# Problema 1

Considere un juego de disparejo entre cuatro jugadores, en el que cada jugador adopta la siguiente estrategia:

    - El jugador 1 siempre escoge al azar entre las dos opciones.
    - El jugador 2 escoge siempre hacia abajo sin importar lo que haya ocurrido anteriormente.
    - El jugador 3 escoge la última opción ganadora de las partidas anteriores. En la primera jugada, escoge arriba.
    - El jugador 4 escoge aquello opuesto al jugador 1 en la última partida. En la primera jugada, escoge al azar.

Utilice AgentPy para generar una simulación del juego, y simule 1000 juegos consecutivos. Muestre los resultados obtenidos para cada uno de los jugadores al final de la simulación.

¿Qué jugador obtuvo mejores resultados? ¿Alguna de las estrategias fue mejor que las otras?


In [ ]:
#Model
import agentpy as ap

# Visualization
import seaborn as sns
import IPython

#### Agent definition

In [ ]:
class Player(ap.Agent):

    def setup(self):

